In [1]:
import gym

env = gym.make("LunarLander-v2")
env.seed(0)

D:\ProgramData\Anaconda3\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


[0]

In [2]:
from ppo import PPO
from shared_ppo import SharedPPO

actor = PPO(env, lam=0.95)

D:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [3]:
def get_experiences(env, actor, episodes=50):
    for episode in range(0, episodes):
        obs = env.reset()
        done = False
        while not done:
            action = actor.act(obs)
            obs2, reward, done, _ = env.step(action)
            actor.remember(obs, action, reward, obs2, done)
            obs = obs2

In [4]:
import numpy as np

def train(env, actor, episodes=100):
    returns = []
    mean = 0
    for episode in range(1, episodes+1):
        obs = env.reset()
        score = 0
        done = False
        while not done:
            action, probs = actor.act(obs)
            obs2, reward, done, _ = env.step(action)
            actor.remember(obs, action, probs, reward, obs2, done)
            actor.train(256)
            obs = obs2
            score += reward
            if done:
                if episode % 100 == 99:
                    actor.epsilon = 0.2
                returns.append(score)
                mean = np.mean(returns[-10:])
                if episode % 10 == 0:
                    print("Episode: " + str(episode) + ", Score: " + str(score) + " Mean Score: " + str(mean))
                if mean >= 200:
                    return returns
    
    return returns

In [5]:
# get_experiences(env, actor)

In [6]:
returns = train(env, actor, episodes=700)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Episode: 10, Score: -51.767962395032576 Mean Score: -180.5470467876246
Episode: 20, Score: -450.5638609330888 Mean Score: -165.20539914427826
Episode: 30, Score: -200.89209640546932 Mean Score: -130.63253844626016
Episode: 40, Score: -62.43354670516338 Mean Score: -136.2800718169114
Episode: 50, Score: -38.45616826694283 Mean Score: -91.23859777767352
Episode: 60, Score: -149.66643399731063 Mean Score: -71.59905382335607
Episode: 70, Score: -28.676708597574702 Mean Score: -53.0879700630079
Episode: 80, Score: -132.62596306935052 Mean Score: -56.68414318868427
Episode: 90, Score: 75.94415864655574 Mean Score: 1.3152719617362891
Episode: 100, Score: -57.263509843765576 Mean Score: -13.286250053436067
Episode: 110, Score: -57.08258383316475 Mean Score: -8.761237359641493
Episode: 120, Score: 104.134815797074 Mean Score: -10.008849336476592
Episode: 130, Score: -142.6162702851759 Mean Score: -41.9

In [7]:
import imageio
import numpy as np

def run_episode(env, model, render=False, record=False):
    images = []
    obs = env.reset()
    done = False
    score = 0
    while not done:
        if record:
            images.append(env.render(mode='rgb_array'))
        if render:
            env.render()
        action, probs = model.act(obs)
        obs, reward, done, info = env.step(action)
        score += reward
        if done:
            print("Score: " + str(score))
            env.close()
            
    if record:
        imageio.mimsave('img/LunarLander.gif', [np.array(img) for i, img in enumerate(images) if i%2 == 0], fps=29)

In [32]:
run_episode(env, actor, render=True, record=True)

Score: 270.74138368275896
